In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Store csv created in part one into a DataFrame
file_path = "../WeatherPy/output_data/cities.csv"
cities_data = pd.read_csv(file_path)
cities_data.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.60,-73.97,79.00,66,44,4.05,US,1597687692
1,Kavieng,-2.57,150.80,81.75,79,9,10.45,PG,1597687753
2,Flin Flon,54.77,-101.86,60.80,72,75,12.75,CA,1597687753
3,Ushuaia,-54.80,-68.30,33.80,86,75,4.68,AR,1597687538
4,Broome,42.25,-75.83,78.01,68,75,6.93,US,1597687753


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Heatmap of humidity
gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '400px',
    'height': '420px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(center=(30.0, 31.0), zoom_level=2)
locations = cities_data[['Lat', 'Long']]
weight = cities_data['Humidity']
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity = 300, point_radius = 5)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.
* Add a "Hotel Name" and "Hotel Rating" columns to the DataFrame.

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
hotel_df = cities_data.loc[(cities_data["Max Temp"] > 70) & (cities_data["Max Temp"] < 80) \
                        & (cities_data["Wind Speed"] < 10) \
                        & (cities_data["Cloudiness"] == 0 )].dropna()

# add empty columns "Hotel Name" and "Hotel Rating"
hotel_df['Hotel Name'] = ''
hotel_df['Hotel Rating'] = ''

# show df
hotel_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
150,Angoche,-16.23,39.91,73.20,82,0,9.17,MZ,1597687777,,
180,Swan River,52.11,-101.27,74.34,58,0,1.52,CA,1597687782,,
185,Matsuura,33.33,129.72,77.00,74,0,4.68,JP,1597687783,,
205,Jining,35.41,116.58,79.27,92,0,6.89,CN,1597687786,,
261,Três Passos,-27.46,-53.93,75.00,70,0,3.00,BR,1597687795,,


### Hotel Map
* Store into variable named `hotel_df`.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# set up base url for API call
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

# Set parameters to search for a hotel
params = {
    'type': 'lodging',
    'radius': 5000,
    'key': g_key
}

# get a response
resp = requests.get(url, params = params)
resp

<Response [200]>

In [8]:
# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row['Lat']
    lng = row['Long']

    params['location'] = f"{lat},{lng}"
    
    print(f"Info: Retrieving results for hotels in city of: {row['City']} ({row['Country']}).")

    # make request, print url and convert to json
    resp = requests.get(url, params = params)
    data = resp.json()
    results = data['results']

    # Grab the first hotel  from the results and store the name and rating
    try:
        print(f"Closest hotel is {results[0]['name']}. Guest rating is: {results[0]['rating']} stars")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Hotel Rating'] = results[0]['rating']

    # handle error
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = ('No Hotel Available')
        print(f'Missing field/result for hotel... skipping this one.')

Info: Retrieving results for hotels in city of: Angoche (MZ).
Closest hotel is Guest House HESADA. Guest rating is: 3.5 stars
Info: Retrieving results for hotels in city of: Swan River (CA).
Closest hotel is Super 8 by Wyndham Swan River MB. Guest rating is: 4 stars
Info: Retrieving results for hotels in city of: Matsuura (JP).
Closest hotel is 山福旅館. Guest rating is: 2.3 stars
Info: Retrieving results for hotels in city of: Jining (CN).
Missing field/result for hotel... skipping this one.
Info: Retrieving results for hotels in city of: Três Passos (BR).
Closest hotel is Hotel Imperial. Guest rating is: 4.2 stars
Info: Retrieving results for hotels in city of: Stubbekøbing (DK).
Closest hotel is Under Valnødden. Guest rating is: 4.8 stars
Info: Retrieving results for hotels in city of: Kumul (CN).
Missing field/result for hotel... skipping this one.
Info: Retrieving results for hotels in city of: Villa María (AR).
Closest hotel is Hotel Milenium. Guest rating is: 4.2 stars
Info: Retriev

In [9]:
# show hotel data frame
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
150,Angoche,-16.23,39.91,73.20,82,0,9.17,MZ,1597687777,Guest House HESADA,3.5
180,Swan River,52.11,-101.27,74.34,58,0,1.52,CA,1597687782,Super 8 by Wyndham Swan River MB,4
185,Matsuura,33.33,129.72,77.00,74,0,4.68,JP,1597687783,山福旅館,2.3
205,Jining,35.41,116.58,79.27,92,0,6.89,CN,1597687786,No Hotel Available,
261,Três Passos,-27.46,-53.93,75.00,70,0,3.00,BR,1597687795,Hotel Imperial,4.2
269,Stubbekøbing,54.89,12.04,73.99,51,0,5.01,DK,1597687796,Under Valnødden,4.8
278,Kumul,42.80,93.45,78.40,17,0,8.88,CN,1597687798,No Hotel Available,
296,Villa María,-32.41,-63.24,73.99,18,0,1.99,AR,1597687801,Hotel Milenium,4.2
319,Kalabo,-14.97,22.68,71.73,19,0,2.71,ZM,1597687805,Sikakubete Guest House Kalabo Town Council,3
518,La Rioja,-29.41,-66.85,71.31,22,0,5.68,AR,1597687784,Hotel Libertador,3.3


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Rating</dt><dd>{Hotel Rating}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))